In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/realaudience

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/realaudience


In [ ]:
import keras
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [ ]:
train_df=pd.read_csv('train_preprocess_1.csv')
cate=pd.read_csv('audience_preprocess_1.csv')


##KMODES + Autoencoder

In [ ]:
from kmodes.kmodes import KModes
from kmodes.util.dissim import ng_dissim

In [ ]:
indexfor=train_df[['bid_id','click']]

In [ ]:
df = train_df[['media_id',
 'placement_type',
 'publisher_id',
 'publisher_name',
 'device_os',
 'ssp_id_fs',
 'campaign_id_fs',
 'media_domain_fs',
 'device_os_version_fs',
 'device_language_fs',
 'device_region_fs',
 'advertisement_id_fs',
 'device_connection_type_fs',
 'adset_id_fs',
 'device_carrier_fs',
 'device_city_fs',
 'device_make_fs',
 'device_model_fs']].astype('category')

In [ ]:
advertisement_dataframe=df[['adset_id_fs','advertisement_id_fs','campaign_id_fs','placement_type']]
advertisement_dataframe = advertisement_dataframe.astype('category')
advertisement_dataframe_matrix=advertisement_dataframe.as_matrix()

In [ ]:
kmode = KModes(n_clusters=25, init='Cao', n_init = 3,verbose=2)
clusters =kmode.fit_predict(advertisement_dataframe_matrix)
advertisement_clusters_df=pd.DataFrame(clusters,columns=['advertisement_cluster'])

import pickle

# It is important to use binary access
with open('fs_advertisement_kmode.pickle', 'wb') as f:
    pickle.dump(kmode, f)

In [ ]:
media_dataframe=df[[ 'media_domain_fs', 'media_id']]
media_dataframe = media_dataframe.astype('category')
media_dataframe_matrix=media_dataframe.as_matrix()

In [ ]:
kmode = KModes(n_clusters=25, init='Cao',n_init = 4, verbose=2)
clusters =kmode.fit_predict(media_dataframe_matrix)
media_clusters_df=pd.DataFrame(clusters,columns=['media_cluster'])

# It is important to use binary access
with open('media_kmode.pickle', 'wb') as f:
    pickle.dump(kmode, f)

In [ ]:
publisher_dataframe=df[['publisher_id','publisher_name','ssp_id_fs']]
publisher_dataframe = publisher_dataframe.astype('category')
publisher_dataframe_matrix=publisher_dataframe.as_matrix()

In [ ]:
kmode = KModes(n_clusters=25, init='Cao',n_init=2, verbose=2)
clusters =kmode.fit_predict(publisher_dataframe_matrix)
publisher_clusters_df=pd.DataFrame(clusters,columns=['publisher_cluster'])

# It is important to use binary access
with open('fs_publisher_kmode.pickle', 'wb') as f:
    pickle.dump(kmode, f)

In [ ]:
device_dataframe=df[['device_carrier_fs','device_city_fs','device_connection_type_fs','device_language_fs',
                     'device_model_fs','device_os','device_os_version_fs','device_region_fs','device_make_fs']]
device_dataframe =device_dataframe.astype('category')
device_dataframe_matrix=device_dataframe.as_matrix()

In [ ]:
kmode = KModes(n_clusters=25, init='Cao',n_init = 10, verbose=2)
clusters =kmode.fit_predict(device_dataframe_matrix)
device_clusters_df=pd.DataFrame(clusters,columns=['device_cluster'])

# It is important to use binary access
with open('device_kmode.pickle', 'wb') as f:
    pickle.dump(kmode, f)

In [ ]:
kmodes=pd.concat([indexfor,media_clusters_df,device_clusters_df,advertisement_clusters_df,publisher_clusters_df],axis=1)
kmodes_train = kmodes
kmodes_train_yes = kmodes_train[kmodes_train['click']==1]
kmodes_train_yes2 = kmodes_train_yes.iloc[:200000,]
kmodes_train_yes2 = kmodes_train_yes2.reset_index()
kmodes_train_yes_ae2 = kmodes_train_yes2.drop(['bid_id','click','index'],axis=1)
kmodes_train_yes = kmodes_train_yes.reset_index()
kmodes_train_yes_ae = kmodes_train_yes.drop(['bid_id','click','index'],axis=1)

In [ ]:
import keras
import pandas as pd
import numpy as np
from keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D
from keras.models import Model
from keras.optimizers import RMSprop

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model


In [ ]:
a2 = pd.get_dummies(kmodes_train_yes_ae2.iloc[:,0], prefix="dev")
b2 = pd.get_dummies(kmodes_train_yes_ae2.iloc[:,1], prefix="med")
c2 = pd.get_dummies(kmodes_train_yes_ae2.iloc[:,2], prefix="ad")
d2 = pd.get_dummies(kmodes_train_yes_ae2.iloc[:,3], prefix="pub")

In [ ]:
a = pd.get_dummies(kmodes_train_yes_ae.iloc[:,0], prefix="dev")
b = pd.get_dummies(kmodes_train_yes_ae.iloc[:,1], prefix="med")
c = pd.get_dummies(kmodes_train_yes_ae.iloc[:,2], prefix="ad")
d = pd.get_dummies(kmodes_train_yes_ae.iloc[:,3], prefix="pub")

In [ ]:
train2 = pd.concat([a2,b2,c2,d2],axis=1)
train2['pub_20'] = 0
train2['pub_18'] = 0
train2['pub_19'] = 0
train = pd.concat([a,b,c,d],axis=1)
train['pub_18'] = 0
train['pub_20'] = 0

In [ ]:
kmodes_train_no = kmodes_train[kmodes_train['click']==0]
kmodes_train1_no = kmodes_train1_no.reset_index()
kmodes_train_no_ae = kmodes_train1_no.drop(['index','click','bid_id'],axis=1)

In [ ]:
encoded_out_train = autoencoder_ssp.predict(train)
ssp_id_train_autoen=pd.DataFrame(encoded_out_train)
ssp_id_train_autoen.columns = train.columns
kmodes_train_no = kmodes_train_no.reset_index()
kmodes_train_no_ae = kmodes_train_no.drop(['index','click','bid_id'],axis=1)

In [ ]:
a = pd.get_dummies(kmodes_train_no_ae.iloc[:,0], prefix="dev")
b = pd.get_dummies(kmodes_train_no_ae.iloc[:,1], prefix="med")
c = pd.get_dummies(kmodes_train_no_ae.iloc[:,2], prefix="ad")
d = pd.get_dummies(kmodes_train_no_ae.iloc[:,3], prefix="pub")

In [ ]:
test = pd.concat([a,b,c,d],axis=1)

In [ ]:

input_dim = Input(shape = (100, ))
# DEFINE THE DIMENSION OF ENCODER ASSUMED 5
encoding_dim = 16
# DEFINE THE ENCODER LAYERS
encoded1 = Dense(8, activation = 'relu')(input_dim)
encoded2 = Dense(encoding_dim, activation = 'relu')(encoded1)
# DEFINE THE DECODER LAYERS
decoded1 = Dense(8, activation = 'relu')(encoded2)
decoded2 = Dense(100, activation = 'sigmoid')(decoded1)
# COMBINE ENCODER AND DECODER INTO AN AUTOENCODER MODEL
autoencoder_ssp = Model(input = input_dim, output = decoded2)
# CONFIGURE AND TRAIN THE AUTOENCODER
autoencoder_ssp.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['accuracy'])

autoencoder_ssp.fit(train2,train2, nb_epoch = 13, batch_size = 128, shuffle = True)

In [ ]:
encoded_out_train = autoencoder_ssp.predict(train)
ssp_id_train_autoen=pd.DataFrame(encoded_out_train)
ssp_id_train_autoen.columns = train.columns

In [ ]:
i =0
dev = ssp_id_train_autoen.iloc[:,i:i+25]
dev.columns = range(25)
med = ssp_id_train_autoen.iloc[:,i+25:i+50]
med.columns = range(25)
ad= ssp_id_train_autoen.iloc[:,i+50:i+75]
ad.columns = range(25)
pub= ssp_id_train_autoen.iloc[:,i+75:]
pub.columns = range(25)

In [ ]:
dev = pd.DataFrame(dev.idxmax(axis=1), columns=['dev'])
med = pd.DataFrame(med.idxmax(axis=1), columns=['med'])
ad = pd.DataFrame(ad.idxmax(axis=1), columns=['ad'])
pub = pd.DataFrame(pub.idxmax(axis=1), columns=['pub'])
predict_train = pd.concat([dev, med, ad, pub],axis=1)

predict_train.columns = ["dev",'med','ad','pub']
kmodes_train_yes_ae.columns=["dev",'med','ad','pub']

In [ ]:
encoded_out_test = autoencoder_ssp.predict(test)
ssp_id_test_autoen=pd.DataFrame(encoded_out_test)
ssp_id_test_autoen.columns = test.columns

In [ ]:
i=0
dev = ssp_id_test_autoen.iloc[:,i:i+25]
dev.columns = range(25)
med = ssp_id_test_autoen.iloc[:,i+25:i+50]
med.columns = range(25)
ad= ssp_id_test_autoen.iloc[:,i+50:i+75]
ad.columns = range(25)
pub= ssp_id_test_autoen.iloc[:,i+75:]
pub.columns = range(25)

In [ ]:
dev = pd.DataFrame(dev.idxmax(axis=1), columns=['dev'])
med = pd.DataFrame(med.idxmax(axis=1), columns=['med'])
ad = pd.DataFrame(ad.idxmax(axis=1), columns=['ad'])
pub = pd.DataFrame(pub.idxmax(axis=1), columns=['pub'])
predict_test = pd.concat([dev,med,ad,pub],axis=1)

In [ ]:
predict_test.columns = ["dev",'med','ad','pub']
accuracy_test = np.mean(np.equal(predict_test,kmodes_train_no_ae))

In [ ]:
k = predict_train-kmodes_train_yes_ae
k = k.where(k==0, 1) #0맞다 1틀리다
k = pd.concat([kmodes_train_yes.loc[:,['bid_id','click']], k],axis=1)
kmodes_train_no_ae.columns=["dev",'med','ad','pub']
k['sum']=k['dev']+k['med']+k['ad']+k['pub']

train_df=train_df.merge(k,on='bid_id')

In [ ]:
# saving in json format
json_model = autoencoder_ssp.to_json()
json_file = open('autoencoder_json.json', 'w')
json_file.write(json_model)


# saving model weights
autoencoder_ssp.save_weights('autoencoder_weights.h5')

##Cate merge

In [ ]:
train_df=train_df.merge(cate,how='left',on='device_ifa')
del cate

In [ ]:
na_index = test_df.loc[pd.isna(test_df["cate_answer_1"]), :].index

test_df.loc[test_df.index[na_index], 'cate_answer_1']=22.2131
test_df.loc[test_df.index[na_index], 'cate_answer_2']= 49.3176
test_df.loc[test_df.index[na_index], 'cate_answer_3']= 15.9415
test_df.loc[test_df.index[na_index], 'cate_answer_4']=  51.3909
test_df.loc[test_df.index[na_index], 'cate_answer_5']=  54.7647
test_df.loc[test_df.index[na_index], 'cate_like_1']= 10.2545
test_df.loc[test_df.index[na_index], 'cate_like_2']=2.55067
test_df.loc[test_df.index[na_index], 'cate_like_3']=0.850046
test_df.loc[test_df.index[na_index], 'cate_like_4']= 14.2648
test_df.loc[test_df.index[na_index], 'cate_like_5']= 10.8701

test_df[['age']]=test_df[['age']].fillna('old')
test_df[['gender']]=test_df[['gender']].fillna('U')
test_df[['marry']]=test_df[['marry']].fillna('C')

##XDEEPFM 

In [ ]:
#1 import model
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from deepctr.models import xDeepFM
from deepctr.inputs import SparseFeat, DenseFeat, get_feature_names

In [ ]:
sparse_features = ['device_ifa', 'dayofweek', 'hour', 'media_id',
       'placement_type', 'publisher_id', 'publisher_name', 'device_os',
       'ssp_id_fs', 'campaign_id_fs', 'media_domain_fs',
       'device_os_version_fs', 'device_language_fs', 'device_region_fs',
       'advertisement_id_fs', 'device_connection_type_fs', 'adset_id_fs',
       'device_carrier_fs', 'device_city_fs', 'device_make_fs',
       'device_model_fs', 'gender', 'age', 'marry'] 
data[sparse_features] = data[sparse_features].astype(str)

In [ ]:
##full 일때

import numpy as np
na_index = data.loc[pd.isna(data["cate_answer_1"]), :].index
data.loc[data.index[na_index], 'cate_answer_1']=22.2131
data.loc[data.index[na_index], 'cate_answer_2']= 49.3176
data.loc[data.index[na_index], 'cate_answer_3']= 15.9415   
data.loc[data.index[na_index], 'cate_answer_4']=  51.3909
data.loc[data.index[na_index], 'cate_answer_5']=  54.7647   
data.loc[data.index[na_index], 'cate_like_1']= 10.2545
data.loc[data.index[na_index], 'cate_like_2']= 2.55067
data.loc[data.index[na_index], 'cate_like_3']=0.850046
data.loc[data.index[na_index], 'cate_like_4']=14.2648
data.loc[data.index[na_index], 'cate_like_5']= 10.8701


In [ ]:
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import *
from deepctr.inputs import  SparseFeat, DenseFeat, get_feature_names

#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
import numpy as np

sparse_features = ['device_ifa', 'dayofweek', 'hour','media_id',
       'placement_type', 'publisher_id', 'publisher_name', 'device_os',
       'ssp_id_fs', 'campaign_id_fs', 'media_domain_fs',
       'device_os_version_fs', 'device_language_fs', 'device_region_fs',
       'advertisement_id_fs', 'device_connection_type_fs', 'adset_id_fs',
       'device_carrier_fs', 'device_city_fs', 'device_make_fs',
       'device_model_fs', 'gender', 'age', 'marry'] 
dense_features = [ 'cate_answer_1', 'cate_answer_2', 'cate_answer_3', 'cate_answer_4',
       'cate_answer_5', 'cate_like_1', 'cate_like_2', 'cate_like_3',
       'cate_like_4', 'cate_like_5']
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['click']
'''
test[sparse_features] = test[sparse_features].fillna('-1', )
test[dense_features] = test[dense_features].fillna(0, )
'''

fixlen_feature_columns_train = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4, use_hash=True, dtype='string')
                        for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                        for feat in dense_features]

dnn_feature_columns_train = fixlen_feature_columns_train
linear_feature_columns_train = fixlen_feature_columns_train

feature_names_train= get_feature_names(linear_feature_columns_train + dnn_feature_columns_train)


In [ ]:
#train, val = train_test_split(data, test_size=0.10, random_state = 10)
train_model_input = {name:data[name] for name in feature_names_train}
#test_model_input = {name:val[name] for name in feature_names_train}
#holdout_model_input = {name:test[name] for name in feature_names_test}


# 4.Define Model,train,predict and evaluate
model = xDeepFM(linear_feature_columns_train, dnn_feature_columns_train, task='binary')
model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy'], )

history = model.fit(train_model_input, data[target].values,
                    batch_size=256, epochs=7, verbose=2,validation_split=0.10,  )
#pred_ans = model.predict(test_model_input, batch_size=128)
#print("test LogLoss", round(log_loss(val[target].values, np.float64(pred_ans)), 4))
#print("test AUC", round(roc_auc_score(val[target].values, pred_ans), 4))
'''
submit = model.predict(holdout_model_input, batch_size=64)


submit = pd.DataFrame(submit)
submit = pd.concat([test['bid_id'], submit], axis=1)
submit.to_csv("final_xDeepFM_5_256_7_sub.csv", index=False, header=False)
'''


model.save_weights('xDeepFM_final_256_7_full_0214.h5')
model.load_weights('xDeepFM_final_256_7_full_0214.h5')

#save
from tensorflow.python.keras.models import  save_model,load_model

save_model(model, 'xDeepFM_final_256_7_full_0214.h5')# save_model, same as before